# Senators notebook

Parses datasets/senado/senators_with_memberships-**termid**.json creating / updating the following polare entites:
* Agents
* Memberships (Senate, Parties)
* Organizations (Senate)
* Posts (Senate)
* Roles (Afiliate, Senator)

## 1. Imports and Constants Declaration

In [46]:
import json   # reads scrapped data
import pandas as pd # helps with file management and visualization
from uuid import uuid4 # creates new uuids for Posts

termid = 55 # legislatura

PROV = 'sen'
FILE_SUFFIX = termid
SUB_DIR = '{:}/'.format(termid)
JSON_PATH =  'datasets/senado/'

AGENTS_PATH = 'datasets/slp/agents/'
MEMBERSHIPS_PATH = 'datasets/slp/memberships/'
ORGANIZATIONS_PATH = 'datasets/slp/organizations/'
ROLES_PATH = 'datasets/slp/roles/'
POSTS_PATH = 'datasets/slp/posts/'


SENADO_URI = '81311052-e5b6-46fe-87ba-83865fa0ffb0' 


 ## 2. Getting the Parties

In [47]:
filename = 'parties.csv'
file_path = '{:}{:}'.format(ORGANIZATIONS_PATH, filename)
df_parties = pd.read_csv(file_path, sep= ';', encoding= 'utf-8', index_col=None)
df_parties.set_index('sigla', inplace=True)


parties_d = df_parties['slp:resource_uri'].to_dict()                        
print({k:parties_d[k]
    for k in ['PT', 'PMDB', 'DEM']})

{'PT': '7b2138a6-df61-11e7-baf1-c82a144c0a85', 'PMDB': '7b210cfa-df61-11e7-9864-c82a144c0a85', 'DEM': '7b20ed7e-df61-11e7-bb65-c82a144c0a85'}


## 3. Reads scrapped json

In [48]:
filename = 'senator_with_memberships-{:}.json'.format(FILE_SUFFIX) 
file_path = '{:}{:}'.format(JSON_PATH, filename)
with open(file_path, mode='r') as f:
    senatorsstr = f.read()
f.close()

senators_with_memberships = json.loads(senatorsstr)

## 4. Agents

In [49]:
mapping = {'skos:prefLabel': 'sen:CodigoParlamentar',
  'foaf:name':'sen:NomeCompletoParlamentar',
  'rdfs:label':'sen:NomeParlamentar',
  'agent_resource_uri':'slp:resource_uri'           
}

agents = [] 
for swm_d in senators_with_memberships:
    agent_d = {mapping[k]: swm_d.get(k, None) for k in mapping}
    if agent_d:
        agents.append(agent_d)

filename = 'senators-{:}.csv'.format(PROV) 
file_path = '{:}{:}{:}'.format(AGENTS_PATH, SUB_DIR, filename)
df = pd.DataFrame.from_dict(agents)    
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)
df.head()

,sen:CodigoParlamentar,sen:NomeCompletoParlamentar,sen:NomeParlamentar,slp:resource_uri
0,5573,Abel Rebouças São José,Abel Rebouças,828a0e78-b457-47de-8112-a7cce8d24563
1,739,Ciro Nogueira Lima Filho,Ciro Nogueira,59888f36-89fc-40e9-adf7-e95bb4906eb2
2,5108,José Aparecido dos Santos,Cidinho Santos,94be5f0e-2cc9-488b-bb32-335a44eb4f1b
3,5136,Cesar Antonio de Souza,Cesar Antonio de Souza,40260ab9-9261-46b9-8ab3-3c569dfa8b24
4,5623,Christopher Belchior Goulart,Christopher Goulart,ad36f1e7-f24a-4f51-9f44-c45468e0d4a1


 ## 5. Senate Memberships
 
 ### 5.1 Membership Process

In [50]:
mapping = {
    'skos:prefLabel': 'sen:CodigoMandato',
    'natureza': 'sen:UfParlamentar',
    'legislatura': 'NumeroLegislatura', 
    'startDate': 'DataInicio', 
    'finishDate': 'DataFim',
    'membership_resource_uri':'slp:resource_uri'
}

memberships = [] 
for swm_d in senators_with_memberships:
    member_uri = swm_d['agent_resource_uri'] 
    terms = swm_d['terms']
    if terms:
        for term_d in terms:
            memberships_d = {mapping[k]: term_d.get(k, None) for k in mapping}
            if memberships_d:
                memberships_d['org:member'] = member_uri
                memberships_d['org:role'] = str(uuid4())
                memberships.append(memberships_d)



df = pd.DataFrame.from_dict(memberships)    
df.head()            

,DataFim,DataInicio,NumeroLegislatura,org:member,org:role,sen:CodigoMandato,sen:UfParlamentar,slp:resource_uri
0,2019-01-31,2015-02-01,55,828a0e78-b457-47de-8112-a7cce8d24563,d7c29205-c2c2-4ef6-ac7d-2c22ae5162dc,492,BA,61e28c9c-c93b-40fc-a73d-3a963632df24
1,2023-01-31,2019-02-01,56,828a0e78-b457-47de-8112-a7cce8d24563,2be63c40-20a5-45f5-8aa6-6a6a023f6a31,492,BA,669716c0-cf09-4d77-896b-8497c3943b34
2,2015-01-31,2011-02-01,54,59888f36-89fc-40e9-adf7-e95bb4906eb2,b6f3d57b-ee59-4176-b1e0-1a0f2a505a34,458,PI,869d897d-3ab6-448a-ba58-8c18a2aa4421
3,2019-01-31,2015-02-01,55,59888f36-89fc-40e9-adf7-e95bb4906eb2,b98e17a7-bf86-4b3f-953a-2255453b45f8,458,PI,155310d6-40c8-4c04-9234-6f312193b877
4,2015-01-31,2011-02-01,54,94be5f0e-2cc9-488b-bb32-335a44eb4f1b,26345664-92f0-4071-9ebb-0cdb93db7952,456,MT,3e9c9056-8f75-4d44-b5a3-da45f12caa33


 ### 5.2 Save memberships

In [51]:
filename = 'memberships_with_senate-{:}.csv'.format(PROV, FILE_SUFFIX) 
file_path = '{:}{:}{:}'.format(MEMBERSHIPS_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)

 ## 6. Roles Deputy
 
 ### 6.1 Roles labeling

In [52]:
df['org:role'].values
df = df['org:role'].to_frame()
df = df.rename({'org:role':'slp:resource_uri'}, axis='columns')
df['rdfs:label'] = 'Deputado'
df.head()

,slp:resource_uri,rdfs:label
0,d7c29205-c2c2-4ef6-ac7d-2c22ae5162dc,Deputado
1,2be63c40-20a5-45f5-8aa6-6a6a023f6a31,Deputado
2,b6f3d57b-ee59-4176-b1e0-1a0f2a505a34,Deputado
3,b98e17a7-bf86-4b3f-953a-2255453b45f8,Deputado
4,26345664-92f0-4071-9ebb-0cdb93db7952,Deputado


 ### 6.2 Roles save

In [53]:
filename = 'senator_roles-{:}.csv'.format(PROV, FILE_SUFFIX) 
file_path = '{:}{:}{:}'.format(ROLES_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)

## 7. Posts

### 7.1 Posts Process

In [54]:
df = df.rename({'slp:resource_uri':'org:role'}, axis='columns')
df['slp:resource_uri'] = [str(uuid4()) for _ in range(df.shape[0])]
df.drop('rdfs:label', axis=1, inplace=True)
df.head()

,org:role,slp:resource_uri
0,d7c29205-c2c2-4ef6-ac7d-2c22ae5162dc,4784aed6-8e21-46ff-86a1-d1be1ad43749
1,2be63c40-20a5-45f5-8aa6-6a6a023f6a31,c09cfee9-c6f5-4ad3-a213-c90d3230312b
2,b6f3d57b-ee59-4176-b1e0-1a0f2a505a34,367f551b-7c06-4f03-82d0-eb4a04461a88
3,b98e17a7-bf86-4b3f-953a-2255453b45f8,d73a5836-2bba-4b41-b557-083d16602b09
4,26345664-92f0-4071-9ebb-0cdb93db7952,ad488278-f8c0-49d9-857c-b6bdf317364d


### 7.2 Save Posts posted in Senado 

In [55]:
filename = 'senator_posts-{:}.csv'.format(PROV, FILE_SUFFIX) 
file_path = '{:}{:}{:}'.format(POSTS_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)

 ## 8. Party Memberships

 ### 8.1 Membership Process

In [56]:
mapping = {
    'sigla': 'sen:SiglaPartido',
    'startDate': 'sen:DataFiliacao' ,
    'finishDate': 'sen:DataDesfiliacao',
    'role_resource_uri': 'org:role',
    'resource_uri': 'slp:resource_uri'
}

memberships = [] 
for swm_d in senators_with_memberships:
    affiliations = swm_d['affiliations']
    if affiliations:
        for affiliation_d in affiliations:
            if affiliation_d['sigla'] in parties_d:
                memberships_d = {mapping[k]: affiliation_d.get(k, None) for k in mapping}
                if memberships_d:
                    memberships_d['org:member'] = member_uri
                    memberships_d['org:post'] = str(uuid4())
                    memberships_d['org:postIn'] = parties_d[affiliation_d['sigla']]
                    memberships.append(memberships_d)



df = pd.DataFrame.from_dict(memberships)    
df.head()            

,org:member,org:post,org:postIn,org:role,sen:DataDesfiliacao,sen:DataFiliacao,sen:SiglaPartido,slp:resource_uri
0,85e8b745-786b-4782-a8a1-e3b3164a3f22,91899a3b-f09e-4e4e-894d-e4cf9490f8b7,7b210070-df61-11e7-97e9-c82a144c0a85,ebf7f8f0-1b0a-4662-9ed8-c9bb19936ec0,None,2015-02-01,PDT,0577ae6b-9fa4-4ff0-ac19-9dd5cd3598d1
1,85e8b745-786b-4782-a8a1-e3b3164a3f22,b8476d7a-3572-45e2-b325-be19454b6c90,7b21152e-df61-11e7-bdcf-c82a144c0a85,ebf7f8f0-1b0a-4662-9ed8-c9bb19936ec0,None,2004-02-14,PP,b2ec603a-26b8-4e59-8a59-624d2f0973ac
2,85e8b745-786b-4782-a8a1-e3b3164a3f22,4bad4bb2-6b4b-4b0f-b822-0062c20fb11a,7b21046c-df61-11e7-8094-c82a144c0a85,ebf7f8f0-1b0a-4662-9ed8-c9bb19936ec0,2004-02-13,1999-02-01,PFL,3cf1400f-9fe7-4552-99ac-d0a4a52a1510
3,85e8b745-786b-4782-a8a1-e3b3164a3f22,4faa95b6-5729-4a66-8077-f41f6e926913,7b211a38-df61-11e7-b1b8-c82a144c0a85,ebf7f8f0-1b0a-4662-9ed8-c9bb19936ec0,None,2011-02-03,PR,da9f5ea7-2636-4240-af94-94e0df597ae9
4,85e8b745-786b-4782-a8a1-e3b3164a3f22,77faa00e-22ad-4db0-ba36-5915aa04b83d,7b20ed7e-df61-11e7-bb65-c82a144c0a85,ebf7f8f0-1b0a-4662-9ed8-c9bb19936ec0,None,2011-02-03,DEM,89a85f62-3dce-499c-a72a-0917f5bf5240


 ### 6.2 Membership Save

In [57]:
filename = 'memberships_with_parties-{:}.csv'.format(PROV) 
file_path = '{:}{:}{:}'.format(MEMBERSHIPS_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)